In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
#import cartopy 
import astropy 
from PIL import Image
import math
from vpython import *

<IPython.core.display.Javascript object>

In [10]:
lat=39.938112
lon=32.655643

a= 7171.8 
e = 0.02
i = 60 
RAAN = 45.916  #3h 03m 40s
w = 10 
true_anomaly = 120

In [11]:
## Define elementary rotation matrices and reflection matrices

"""
Rotation about x axis
"""
def getR1(alpha):
    return np.matrix(
        [
            [1.0,0,0],
            [0,np.cos(alpha),np.sin(alpha)],
            [0,-np.sin(alpha),np.cos(alpha)],
        ]
    )


"""
Rotation about y axis
"""
def getR2(alpha):
    return np.matrix(
        [
            [np.cos(alpha),0, -np.sin(alpha)],
            [0,1.0,0],
            [np.sin(alpha),0, np.cos(alpha)],
        ]
    )

"""
Rotation about z axis
"""
def getR3(alpha):
    return np.matrix(
        [
            [np.cos(alpha),np.sin(alpha),0],
            [-np.sin(alpha),np.cos(alpha),0],
            [0,0,1],

        ]
    )



"""
Reflection about x
"""

def getS1():
    return np.matrix(
        [
            [-1,0,0],
            [ 0,1,0],
            [ 0,0,1],

        ]
    )

"""
Reflection about y
"""
def getS2():
    return np.matrix(
        [
            [ 1,0,0],
            [ 0,-1,0],
            [ 0,0,1],

        ]
    )


"""
Reflection about z
"""
def getS3():
    return np.matrix(
        [
            [ 1,0,0],
            [ 0,1,0],
            [ 0,0,-1],

        ]
    )

In [12]:
## Lets create the observer class

class SphericalEarthModel():
    R = 6371000


class Observer(object):

    def __init__(self, name, lat, lon, height):
        self.name = name
        self.lat = lat
        self.lon = lon
        
        self.rlat = np.radians(lat)
        self.rlon = np.radians(lon)
        
        self.height = height
        
        # geocentric position vector
        self.r_L = np.array(
            [
                (SphericalEarthModel.R+self.height)*np.cos(self.rlat)*np.cos(self.rlon),
                (SphericalEarthModel.R+self.height)*np.cos(self.rlat)*np.sin(self.rlon),
                (SphericalEarthModel.R+self.height)*np.sin(self.rlat),
            ]
        ).reshape((-1,1)) ## as a column vector
        
        self.R_L_to_E = getR3(np.pi-self.rlon)*getR2(np.pi/2-self.rlat)*getS2()
        
        RltoE = getR3(-(np.pi/2+self.rlon))*getR2(-(np.pi/2-self.rlat))
        
        print ("All Equals topocentric to geocentric , :")
        print (self.R_L_to_E)
        print (RltoE)
    
    
    def az_el_to_x(self, azimuth, elevation, dist):
        a = np.radians(azimuth)
        e = np.radians(elevation)
        
        x = np.array([
            dist*np.cos(e)*np.cos(a),
            dist*np.cos(e)*np.sin(a),
            dist*np.sin(e)
        ]).reshape((-1,1)) ## as a column vector
        
        return x
    
    
    def x_to_az_el(self, x):
        
        dist = np.sqrt(x[0,0]**2 + x[1,0]**2+ x[2,0]**2) # sqrt (x^2 + y^2 + z^2)
        
        if (np.allclose(dist,0)):
            print ("Distance is zero !!!! in azimuth conversion")
            
        x = x/dist ## unit vector
        
        zenith = np.arccos(x[2,0])
        elev = np.pi/2 - zenith
        
        azim = np.arctan2(x[1,0],x[0,0])
        
        if (azim <0):
            azim += 2*np.pi
        
        return np.degrees(azim),np.degrees(elev),dist
    
    def convert_to_ECEF(self, x):
        ## rotate direction vector x to ECEF axes
        ## add observer location as a shift
        print (self.R_L_to_E*x.reshape((-1,1)))
        return self.r_L + self.R_L_to_E*x.reshape((-1, 1))
    
    
    def convert_fromECEF(self,X):
        ## subtract observer location to obtain direction vector in ECEF
        ## rotate direction vector to local
        Dx  = X - self.r_L
                
        return self.R_L_to_E.T*Dx
    
    
    def convert_to_lat_lon(self,x):
        dist = np.sqrt(x[0,0]**2 + x[1,0]**2+ x[2,0]**2) # sqrt (x^2 + y^2 + z^2)
        
        if (np.allclose(dist,0)):
            print ("Distance is zero !!!! in azimuth conversion")
                
        colat = np.arccos(x[2,0]/dist)
        lat = np.pi/2 - colat
        
        lon = np.arctan2(x[1,0],x[0,0])
        
        if (lon <0):
            lon += 2*np.pi
        
        return np.degrees(lat),np.degrees(lon),dist
    
    def __str__(self):
        return "Observer at (%3.4f,%3.3f, %6.2f)" % (self.lat,self.lon,self.height)

In [13]:
## Test Observer class


gmtk = Observer("Home", lat, lon,1000)

print (gmtk)


# azimuth of an object at north direction at horizon
x = np.array([1,0,0]).reshape((-1,1))
print (gmtk.x_to_az_el(x)," Shall give azimuth 0 elevation 0")

# slightly move to zenith
x = np.array([1,0,1]).reshape((-1,1))
print (gmtk.x_to_az_el(x)," Shall give azimuth 0 elevation 45")

# slightly move to west
x = np.array([1,-1,1]).reshape((-1,1))
print (gmtk.x_to_az_el(x)," Shall give azimuth 315")


# Do reverse
print (gmtk.az_el_to_x(315.0, 35.264389682754654, 1.7320508075688772)," Shall give 1,-1,1")


## Obtain object coordinates at 3000 meter above zenith
x = gmtk.az_el_to_x(0,90,3000)

## Convert x to geocentric
X = gmtk.convert_to_ECEF(x)

## Lets find latitude and longitude of X

r = np.sqrt(X[0,0]**2 + X[1,0]**2 + X[2,0]**2)
print (r, " Shall be 6371000 + 4000 meters")

## Move X towards north pole by incrementing Z
## we shall see zero azimuth and decreasing elevation

X[2,0] += 10000 # 10 km

x = gmtk.convert_fromECEF(X)

print ("Direction Vector ",x)

print (gmtk.x_to_az_el(x))

All Equals topocentric to geocentric , :
[[-0.54048442 -0.53958868  0.64553904]
 [-0.34639424  0.84192877  0.41372331]
 [ 0.7667383   0.          0.64195979]]
[[-0.34639424 -0.84192877 -0.41372331]
 [ 0.54048442 -0.53958868  0.64553904]
 [-0.7667383   0.          0.64195979]]
Observer at (39.9381,32.656, 1000.00)
(0.0, 0.0, 1.0)  Shall give azimuth 0 elevation 0
(0.0, 44.99999999999999, 1.4142135623730951)  Shall give azimuth 0 elevation 45
(315.0, 35.264389682754654, 1.7320508075688772)  Shall give azimuth 315
[[ 1.]
 [-1.]
 [ 1.]]  Shall give 1,-1,1
[[1936.61710989]
 [1241.16992937]
 [1925.87937735]]
6375000.0  Shall be 6371000 + 4000 meters
Direction Vector  [[ 7.66738303e+03]
 [-2.23810120e-10]
 [ 9.41959792e+03]]
(359.9999999999983, 50.85503147425922, 12145.681847757947)


In [14]:
class Satellite(object):
    
    GM = 3986004.418e8
    
    def __init__(self,a,e,i,ran,w,T0):
        self.a = a # semi major axis in meters
        self.i = i # inclination in degrees
        self.e = e # eccentricity
        self.ran = ran # Right ascension of the ascending node in degrees
        self.w = w # argument of perigee in degrees
        self.T0 = T0 # perigee passage time
        self.T = 2*np.pi*np.sqrt(self.a**3/Satellite.GM)
        self.n = np.sqrt(Satellite.GM/self.a**3)
        
        
        self.R_o_to_I = getR3(-np.radians(self.ran))*getR1(-np.radians(self.i))*getR3(-np.radians(self.w))
        
        
    
    """
    given time t, return satellite position
    """
    def pos(self, t):
        
        dt = (t.tt-self.T0).sec ## Use terrestrial time for time difference
        
#        dt = np.mod(dt,self.T)
        
        M = self.n*(dt)
        
        E = M
        E_new = E
        
        i =0 ## iteration
        while(True and i<5):
            E_new = M+self.e*np.sin(E)
            
            if (np.abs(E_new-E)<1e-10):
                E = E_new
                ## Early break with precision
                break
                
            E = E_new
            i = i +1
        
        sv = np.sqrt(1-self.e**2)*np.sin(E)
        cv = np.cos(E)-self.e
        
        v = np.arctan2(sv,cv)
        
        if (v<0):
            v+=2*np.pi
        
        r = self.a*(1-self.e**2)/(1+self.e*np.cos(v))
        
        r_o = np.array([
            r*np.cos(v),
            r*np.sin(v),
            0
        ]).reshape((-1,1))
        
        r_i = self.R_o_to_I*r_o
        
        gast = t.sidereal_time("mean",longitude=0).rad
        
        R_gast = getR3(gast)
        
#        ECI -> ECEF
        r_e = R_gast*r_i
        
        return r_i, r_e

In [15]:
from astropy.time import Time,TimeDelta

T0 = Time("2012-02-02T13:15:00", format='isot', scale='tt')

gmtk = Observer("Home", lat, lon,1000)


geos = Satellite(a*(10**3), e, i, RAAN, w, T0)


pos = geos.pos(T0)
x = gmtk.convert_fromECEF(pos[1])
azel = gmtk.x_to_az_el(x)


print('ECEF:', pos[0])
print('ECI:', pos[1])
print ("Azimuth and Elevation : ",azel[0],azel[1])


All Equals topocentric to geocentric , :
[[-0.54048442 -0.53958868  0.64553904]
 [-0.34639424  0.84192877  0.41372331]
 [ 0.7667383   0.          0.64195979]]
[[-0.34639424 -0.84192877 -0.41372331]
 [ 0.54048442 -0.53958868  0.64553904]
 [-0.7667383   0.          0.64195979]]
ECEF: [[4377091.26443575]
 [5396464.19980117]
 [1056951.61647098]]
ECI: [[1166231.0919187 ]
 [6849865.60723475]
 [1056951.61647098]]
Azimuth and Elevation :  113.11146384213482 -20.66279801899909


###QUESTİON 2 ###

In [20]:
import pandas as pd

# necessary classical orbit elements
mass = 1 # in kg 
eccentricity = 0.02 # unitless
semimajor_axis = 7171.8
true_anomaly = 120 #true anomaly
mu = 3.986*(10**5) 

def distance():

    distance = []
    for i in range(true_anomaly, 360+1):
        R = (semimajor_axis*(1-eccentricity**2))/(1+ eccentricity*math.cos(i))
        distance.append(R)
    
    return distance

def potential():
    
    R = distance()
    potential = []
    for i in R:
        e_p = ((-((mass*mu)/i))*(1e-3))#energy potential
        potential.append(e_p)
    
    return potential
        
def kinetic():
    
    R = distance()
    kinetic = []
    for i in R:
        V = ((2*((mu/i)+(-mu/(2*semimajor_axis))))**(1/2))
        e_k = ((1/2)*(mass)*(V**2))*(1e-3)

        kinetic.append(e_k)#energy kinetic
    
    return kinetic


x = dist()
y = potential()
z = kinetic()

df = pd.DataFrame()
df['R (Distance)'] = x
df['Potential_E'] = y
df['Kinetic_E'] = z
df['Mechanic_E'] = df['Potential_E'] + df['Kinetic_E']
print(df)

     R (Distance)  Potential_E  Kinetic_E  Mechanic_E
0     7054.065561    -0.056506   0.028717   -0.027789
1     7175.915399    -0.055547   0.027758   -0.027789
2     7295.399524    -0.054637   0.026848   -0.027789
3     7298.548971    -0.054614   0.026824   -0.027789
4     7182.258133    -0.055498   0.027708   -0.027789
..            ...          ...        ...         ...
236   7247.241631    -0.055000   0.027211   -0.027789
237   7109.753355    -0.056064   0.028274   -0.027789
238   7029.742829    -0.056702   0.028913   -0.027789
239   7076.425304    -0.056328   0.028538   -0.027789
240   7209.838620    -0.055286   0.027496   -0.027789

[241 rows x 4 columns]
